In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Packages:

In [65]:
!pip install stellargraph
import stellargraph

In [66]:
!pip install keras.utils
import keras.utils

In [67]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, AveragePooling2D, BatchNormalization
from keras.applications.vgg16 import VGG16
from keras.callbacks import Callback

import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import DeepGraphCNN
from stellargraph import StellarGraph
from stellargraph import datasets

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv1D, MaxPool1D, Dropout, Flatten
from tensorflow.keras.losses import binary_crossentropy
import tensorflow as tf

Read Data from file

In [68]:
 def _load_from_txt_file(filename, names=None, dtype=None, index_increment=None):
   path = ('/content/drive/MyDrive/M.Phil/Brain_graphs1/' + filename)
   df = pd.read_csv(path, header=None, index_col=False, dtype=dtype, names=names)
   if index_increment:
     df.index = df.index + index_increment
   return df


In [69]:
df_graph = _load_from_txt_file(filename="Brain_A.csv", names=["source", "target"])
print(df_graph)
df_graph_ids = _load_from_txt_file(filename="Brain_graph_indicator.csv", names=["graph_id"], index_increment=1)
print(df_graph_ids)
df_node_labels = _load_from_txt_file(filename="Brain_node_labels.csv", dtype="category",names=["node_id"], index_increment=1)
print(df_node_labels)
df_node_features = pd.get_dummies(df_node_labels)
#print(df_node_features)
df_graph_labels = _load_from_txt_file(filename="Brain_graph_labels.csv", dtype="category", names=["class_id"], index_increment=1)
print(df_graph_labels)

        source  target
0            7       1
1           14       1
2           21       1
3           23       1
4           24       1
...        ...     ...
137511   12955   12960
137512   12956   12960
137513   12957   12960
137514   12958   12960
137515   12959   12960

[137516 rows x 2 columns]
       graph_id
1             1
2             1
3             1
4             1
5             1
...         ...
12956       162
12957       162
12958       162
12959       162
12960       162

[12960 rows x 1 columns]
      node_id
1           1
2           2
3           3
4           4
5           5
...       ...
12956      76
12957      77
12958      78
12959      79
12960      80

[12960 rows x 1 columns]
    class_id
1          1
2          1
3          1
4          1
5          1
..       ...
158        0
159        0
160        0
161        0
162        0

[162 rows x 1 columns]


In [70]:
def graph_for_nodes(nodes):
  edges = df_graph[df_graph["source"].isin(nodes.index)]
  return StellarGraph(nodes, edges)

In [71]:
  groups = df_node_features.groupby(df_graph_ids["graph_id"])
  graphs = [graph_for_nodes(nodes) for _, nodes in groups]
  print(graphs[0].info())
  print(" ")
  print(graphs[1].info())
  print(" ")
  print(graphs[160].info())
  print(" ")
  print(graphs[161].info())

StellarGraph: Undirected multigraph
 Nodes: 80, Edges: 854

 Node types:
  default: [80]
    Features: float32 vector, length 80
    Edge types: default-default->default

 Edge types:
    default-default->default: [854]
        Weights: all 1 (default)
        Features: none
 
StellarGraph: Undirected multigraph
 Nodes: 80, Edges: 852

 Node types:
  default: [80]
    Features: float32 vector, length 80
    Edge types: default-default->default

 Edge types:
    default-default->default: [852]
        Weights: all 1 (default)
        Features: none
 
StellarGraph: Undirected multigraph
 Nodes: 80, Edges: 948

 Node types:
  default: [80]
    Features: float32 vector, length 80
    Edge types: default-default->default

 Edge types:
    default-default->default: [948]
        Weights: all 1 (default)
        Features: none
 
StellarGraph: Undirected multigraph
 Nodes: 80, Edges: 996

 Node types:
  default: [80]
    Features: float32 vector, length 80
    Edge types: default-default->defa

In [72]:
summary = pd.DataFrame(
    [(g.number_of_nodes(), g.number_of_edges()) for g in graphs],
    columns=["nodes", "edges"],
)
summary.describe().round(1)

,nodes,edges
count,162.0,162.0
mean,80.0,848.9
std,0.0,125.2
min,80.0,326.0
25%,80.0,809.0
50%,80.0,872.0
75%,80.0,932.0
max,80.0,1026.0


In [73]:
graph_labels = df_graph_labels["class_id"]
graph_labels.value_counts().to_frame()

,class_id
0,100
1,62


In [74]:
graph_labels = pd.get_dummies(graph_labels, drop_first=True)

In [75]:
generator = PaddedGraphGenerator(graphs=graphs)

In [102]:
train_graphs, test_graphs = model_selection.train_test_split(
    graph_labels, train_size=0.8, test_size=0.2, stratify=graph_labels, shuffle=True
)
print(graph_labels)

     1
1    1
2    1
3    1
4    1
5    1
..  ..
158  0
159  0
160  0
161  0
162  0

[162 rows x 1 columns]


In [103]:
gen = PaddedGraphGenerator(graphs=graphs)

train_gen = gen.flow(
    list(train_graphs.index - 1),
    targets=train_graphs.values,
    batch_size=50,
    symmetric_normalization=False,
)

test_gen = gen.flow(
    list(test_graphs.index - 1),
    targets=test_graphs.values,
    batch_size=1,
    symmetric_normalization=False,
)

In [110]:
 k = 35 # the number of rows for the output tensor 
 layer_sizes = [32, 32, 32, 1]
 dgcnn_model = DeepGraphCNN(
    layer_sizes=layer_sizes,
    activations=["tanh", "tanh", "tanh", "tanh"],
    k=k,
    bias=False,
    generator=generator,
)
x_inp, x_out = dgcnn_model.in_out_tensors()

In [111]:
x_out = Conv1D(filters = 32, kernel_size = sum(layer_sizes),strides=sum(layer_sizes))(x_out)
x_out = MaxPool1D(pool_size=2)(x_out)
x_out = Conv1D(filters = 64, kernel_size = 5, strides=1)(x_out)
x_out = MaxPool1D(pool_size=2)(x_out)
x_out = Conv1D(filters = 128, kernel_size = 5, strides=1)(x_out)
x_out = Flatten()(x_out)
x_out = Dense(256, activation='relu')(x_out)
x_out = Dropout(0.5)(x_out)
x_out = Dense(128, activation='relu')(x_out)
x_out = Dropout(0.5)(x_out)
predictions = Dense(1, activation='sigmoid')(x_out)

In [112]:
model = Model(inputs=x_inp, outputs=predictions)

model.compile(
    optimizer = Adam(lr=0.001), loss=binary_crossentropy, metrics=["acc"],
)
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_37 (InputLayer)          [(None, None, 80)]   0           []                               
                                                                                                  
 dropout_70 (Dropout)           (None, None, 80)     0           ['input_37[0][0]']               
                                                                                                  
 input_39 (InputLayer)          [(None, None, None)  0           []                               
                                ]                                                                 
                                                                                                  
 graph_convolution_48 (GraphCon  (None, None, 32)    2560        ['dropout_70[0][0]',      

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [113]:
def train_and_evaluate__model(model, train_gen, labels_train, test_gen, labels_test):
  # callback = [TerminateOnBaseline(monitor='accuracy', baseline=1)]
  # callback = myCallback()
  # print("Train Labels",labels_train)
  model.fit(train_gen, epochs=100, batch_size=64)
  #prediticted_classes = model.predict_classes(data_test, batch_size=4)
  prediticted_classes = (model.predict(test_gen, batch_size=4) > 0.5).astype("int32")
  print("Predict Classes", prediticted_classes)
  print("Evaluate on test data")
  results = model.evaluate(test_gen, verbose=0)
  print("test loss, test acc:", results)
  return results[1]

In [115]:
accuracy = train_and_evaluate__model(model, train_gen, graph_labels, test_gen, graph_labels)
print(accuracy)


Epoch 1/100
3/3 [==============================] - 0s 39ms/step - loss: 7.0145e-04 - acc: 1.0000
Epoch 2/100
3/3 [==============================] - 0s 40ms/step - loss: 9.8846e-04 - acc: 1.0000
Epoch 3/100
3/3 [==============================] - 0s 39ms/step - loss: 6.4215e-04 - acc: 1.0000
Epoch 4/100
3/3 [==============================] - 0s 42ms/step - loss: 0.0021 - acc: 1.0000
Epoch 5/100
3/3 [==============================] - 0s 46ms/step - loss: 5.7755e-04 - acc: 1.0000
Epoch 6/100
3/3 [==============================] - 0s 61ms/step - loss: 1.6066e-04 - acc: 1.0000
Epoch 7/100
3/3 [==============================] - 0s 49ms/step - loss: 0.0010 - acc: 1.0000
Epoch 8/100
3/3 [==============================] - 0s 52ms/step - loss: 6.5913e-04 - acc: 1.0000
Epoch 9/100
3/3 [==============================] - 0s 54ms/step - loss: 1.2881e-04 - acc: 1.0000
Epoch 10/100
3/3 [==============================] - 0s 39ms/step - loss: 1.0742e-04 - acc: 1.0000
Epoch 11/100
3/3 [===================

*Clear* Model

In [ ]:
keras.backend.clear_session()